In [ ]:
import cx_Oracle
import time, datetime
from sklearn.externals import joblib
import pandas as pd
import gc
import os

In [ ]:
class OracleDB:

    def __init__(self, user, password, sid):
        self.user = user
        self.password = password
        self.sid = sid
        self.DatabaseError = cx_Oracle.DatabaseError

    def connect(self):
        self.connection = cx_Oracle.connect(self.user, self.password, self.sid)
        self.cursor = self.connection.cursor()

    def close(self):
        self.cursor.close()
        self.connection.close()

In [ ]:
def read_from_db(db, columns, file_name, append_header):
    index, cols = [], []
    rows = db.cursor.fetchmany()
    if not rows: return False
    index.extend([x[0] for x in rows])
    cols.extend([x[1:] for x in rows])

    data = pd.DataFrame(cols, index=index, columns=columns[1:])
    data.to_csv(file_name, mode='a', compression='gzip', header=append_header, sep=';',  encoding='cp1251')
    append_header=False

    return True

def get_balance(db, sql, file_name):
    print('Connecting...')
    db.connect()
    
    start_time = datetime.datetime.now()
    print('Getting data ... ')
    db.cursor.arraysize = 10_000

    db.cursor.execute(sql)
    columns=[x[0] for x in db.cursor.description]
    index, cols = [], []
    start_time = time.time()
    append_header = True
    
    if os.path.exists(file_name):
        os.remove(file_name)
        
    count = 0
    while True:
        result = read_from_db(db, columns, file_name, append_header)
        count += 1
        append_header = False
        if not result: break
        print('Downloaded {:,} lines, {:7.0f}sec. passed'.format(count * db.cursor.arraysize, time.time() - start_time))
    
    db.close()

In [ ]:
db4 = OracleDB('', '', '')
file_name='cvm_churn_fot_zp_agr.csv'
sql='''select /*+parallel(6)*/* from cvm_churn_fot_zp_agr'''

get_balance(db4, sql, file_name)

# Файл сохраняется с сжатием gzip
# Для чтения необходим параметр compression='gzip'

In [ ]:
df = pd.read_csv(file_name, compression='gzip', sep=';')